In [1]:
import numpy as np
import mne
from scipy import signal
from scipy.interpolate import RectBivariateSpline
from mne.filter import resample, filter_data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from lspopt import spectrogram_lspopt
from matplotlib.colors import Normalize, ListedColormap

import logging
LOGGING_TYPES = dict(DEBUG=logging.DEBUG, INFO=logging.INFO, WARNING=logging.WARNING,
                     ERROR=logging.ERROR, CRITICAL=logging.CRITICAL)
logger = logging.getLogger('yasa')

%matplotlib qt

In [2]:
# Load the EDF file
fname = "P8_N3"  # define here
lr = "L"  # define here
location = f"/Users/amirhosseindaraie/Desktop/data/autoscoring-material/data/Zmax Donders/{fname}"
raw = mne.io.read_raw_edf(f"{location}/EEG {lr}.edf", preload=True, verbose=0)
raw.pick_types(eeg=True)
# fig = raw.plot(use_opengl=False)

# Apply a zero-phase bandpass filter between 0.5 - 45 Hz
raw.filter(0.5, 45)

# Plot properties of the filter
filt = mne.filter.create_filter(raw._data, 256, 0.5, 40)
mne.viz.plot_filter(filt, 256)
plt.savefig('filter shape.png', dpi=100, bbox_inches='tight')

# Extract the data and convert from V to uV
data = raw._data * 1e6
sf = raw.info["sfreq"]
chan = raw.ch_names

# Let's have a look at the data
print("Chan =", chan)
print("Sampling frequency =", sf, "Hz")
print("Data shape =", data.shape)


def format_seconds_to_hhmmss(seconds):
    # Return hhmmss of total seconds parameter
    hours = seconds // (60 * 60)
    seconds %= 60 * 60
    minutes = seconds // 60
    seconds %= 60
    return "%02i:%02i:%02i" % (hours, minutes, seconds)


print(
    f"Duration: {data.shape[1]/sf} (sec) OR {format_seconds_to_hhmmss(data.shape[1]/sf)}"
)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth

In [3]:
import antropy as ant
import scipy.signal as sp_sig
import scipy.stats as sp_stats
from numpy import apply_along_axis as apply

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", -1)

# Time vector in seconds
times = np.arange(data.size) / sf


def sliding_window(data, sf, window, step=None, axis=-1):
    """Calculate a sliding window of a 1D or 2D EEG signal.
    .. versionadded:: 0.1.7
    Parameters
    ----------
    data : numpy array
        The 1D or 2D EEG data.
    sf : float
        The sampling frequency of ``data``.
    window : int
        The sliding window length, in seconds.
    step : int
        The sliding window step length, in seconds.
        If None (default), ``step`` is set to ``window``,
        which results in no overlap between the sliding windows.
    axis : int
        The axis to slide over. Defaults to the last axis.
    Returns
    -------
    times : numpy array
        Time vector, in seconds, corresponding to the START of each sliding
        epoch in ``strided``.
    strided : numpy array
        A matrix where row in last dimension consists of one instance
        of the sliding window, shape (n_epochs, ..., n_samples).
    Notes
    -----
    This is a wrapper around the
    :py:func:`numpy.lib.stride_tricks.as_strided` function.
    Examples
    --------
    With a 1-D array
    >>> import numpy as np
    >>> from yasa import sliding_window
    >>> data = np.arange(20)
    >>> times, epochs = sliding_window(data, sf=1, window=5)
    >>> times
    array([ 0.,  5., 10., 15.])
    >>> epochs
    array([[ 0,  1,  2,  3,  4],
           [ 5,  6,  7,  8,  9],
           [10, 11, 12, 13, 14],
           [15, 16, 17, 18, 19]])
    >>> sliding_window(data, sf=1, window=5, step=1)[1]
    array([[ 0,  1,  2,  3,  4],
           [ 2,  3,  4,  5,  6],
           [ 4,  5,  6,  7,  8],
           [ 6,  7,  8,  9, 10],
           [ 8,  9, 10, 11, 12],
           [10, 11, 12, 13, 14],
           [12, 13, 14, 15, 16],
           [14, 15, 16, 17, 18]])
    >>> sliding_window(data, sf=1, window=11)[1]
    array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])
    With a N-D array
    >>> np.random.seed(42)
    >>> # 4 channels x 20 samples
    >>> data = np.random.randint(-100, 100, size=(4, 20))
    >>> epochs = sliding_window(data, sf=1, window=10)[1]
    >>> epochs.shape  # shape (n_epochs, n_channels, n_samples)
    (2, 4, 10)
    >>> epochs
    array([[[  2,  79,  -8, -86,   6, -29,  88, -80,   2,  21],
            [-13,  57, -63,  29,  91,  87, -80,  60, -43, -79],
            [-50,   7, -46, -37,  30, -50,  34, -80, -28,  66],
            [ -9,  10,  87,  98,  71, -93,  74, -66, -20,  63]],
           [[-26, -13,  16,  -1,   3,  51,  30,  49, -48, -99],
            [-12, -52, -42,  69,  87, -86,  89,  89,  74,  89],
            [-83,  31, -12, -41, -87, -92, -11, -48,  29, -17],
            [-51,   3,  31, -99,  33, -47,   5, -97, -47,  90]]])
    """
    from numpy.lib.stride_tricks import as_strided

    assert axis <= data.ndim, "Axis value out of range."
    assert isinstance(sf, (int, float)), "sf must be int or float"
    assert isinstance(window, (int, float)), "window must be int or float"
    assert isinstance(step, (int, float, type(None))), (
        "step must be int, " "float or None."
    )
    if isinstance(sf, float):
        assert sf.is_integer(), "sf must be a whole number."
        sf = int(sf)
    assert isinstance(axis, int), "axis must be int."

    # window and step in samples instead of points
    window *= sf
    step = window if step is None else step * sf

    if isinstance(window, float):
        assert window.is_integer(), "window * sf must be a whole number."
        window = int(window)

    if isinstance(step, float):
        assert step.is_integer(), "step * sf must be a whole number."
        step = int(step)

    assert step >= 1, "Stepsize may not be zero or negative."
    assert window < data.shape[axis], (
        "Sliding window size may not exceed " "size of selected axis"
    )

    # Define output shape
    shape = list(data.shape)
    shape[axis] = np.floor(data.shape[axis] / step - window / step + 1).astype(int)
    shape.append(window)

    # Calculate strides and time vector
    strides = list(data.strides)
    strides[axis] *= step
    strides.append(data.strides[axis])
    strided = as_strided(data, shape=shape, strides=strides)
    t = np.arange(strided.shape[-2]) * (step / sf)

    # Swap axis: n_epochs, ..., n_samples
    if strided.ndim > 2:
        strided = np.rollaxis(strided, -2, 0)
    return t, strided

# Convert the EEG data to 30-sec data
times, data_win = sliding_window(data[0], sf, window=30)

# Convert times to minutes
times /= 60


def lziv(x):
    """Binarize the EEG signal and calculate the Lempel-Ziv complexity."""
    return ant.lziv_complexity(x > x.mean(), normalize=True)


/var/folders/c4/x03my2dj70s46m109bn58l2m0000gn/T/ipykernel_9211/1829453070.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("max_colwidth", -1)


In [ ]:
# Calculate standard descriptive statistics
hmob, hcomp = ant.hjorth_params(data_win, axis=1)

# Feature extraction
df_feat = {
    # Statistical
    "std": apply(np.std, arr=data_win, axis=1, ddof=1),
    "mean": apply(np.mean, arr=data_win, axis=1),
    "median": apply(np.median, arr=data_win, axis=1),
    "iqr": apply(sp_stats.iqr, arr=data_win, axis=1, rng=(25, 75)),
    "skew": apply(sp_stats.skew, arr=data_win, axis=1),
    "kurt": apply(sp_stats.kurtosis, arr=data_win, axis=1),
    "nzc": apply(ant.num_zerocross, arr=data_win, axis=1),
    "hmob": hmob,
    "hcomp": hcomp,
    # Entropy
    "perm_entropy": apply(ant.perm_entropy, axis=1, arr=data_win, normalize=True),
    "svd_entropy": apply(ant.svd_entropy, 1, data_win, normalize=True),
    "sample_entropy": apply(ant.sample_entropy, 1, data_win),
    "app_entropy": apply(ant.app_entropy, 1, data_win, order=2),
    "spec_entropy": apply(
        ant.spectral_entropy,
        1,
        data_win,
        sf,
        normalize=True,
        method="welch",
        nperseg=50,
    ),
    "lziv": apply(ant.lziv_complexity, 1, data_win),
    # Fractal dimension
    "dfa": apply(ant.detrended_fluctuation, 1, data_win),
    "petrosian": apply(ant.petrosian_fd, 1, data_win),
    "katz": apply(ant.katz_fd, 1, data_win),
    "higuchi": apply(ant.higuchi_fd, 1, data_win),
}


df_feat = pd.DataFrame(df_feat)
df_feat.head()


In [ ]:
from scipy.integrate import simps
from scipy.signal import welch

# Estimate power spectral density using Welch's method
freqs, psd = welch(data_win, sf, nperseg=int(4 * sf))


def bandpower_from_psd_ndarray(
    psd,
    freqs,
    bands=[
        (0.5, 4, "Delta"),
        (4, 8, "Theta"),
        (8, 12, "Alpha"),
        (12, 16, "Sigma"),
        (16, 30, "Beta"),
        (30, 40, "Gamma"),
    ],
    relative=True,
):
    """Compute bandpowers in N-dimensional PSD.
    This is a NumPy-only implementation of the :py:func:`yasa.bandpower_from_psd` function,
    which supports 1-D arrays of shape (n_freqs), or N-dimensional arays (e.g. 2-D (n_chan,
    n_freqs) or 3-D (n_chan, n_epochs, n_freqs))
    .. versionadded:: 0.2.0
    Parameters
    ----------
    psd : :py:class:`numpy.ndarray`
        Power spectral density of data, in uV^2/Hz. Must be a N-D array of shape (..., n_freqs).
        See :py:func:`scipy.signal.welch` for more details.
    freqs : :py:class:`numpy.ndarray`
        Array of frequencies. Must be a 1-D array of shape (n_freqs,)
    bands : list of tuples
        List of frequency bands of interests. Each tuple must contain the lower and upper
        frequencies, as well as the band name (e.g. (0.5, 4, 'Delta')).
    relative : boolean
        If True, bandpower is divided by the total power between the min and
        max frequencies defined in ``band`` (default 0.5 to 40 Hz).
    Returns
    -------
    bandpowers : :py:class:`numpy.ndarray`
        Bandpower array of shape *(n_bands, ...)*.
    """
    # Type checks
    assert isinstance(bands, list), "bands must be a list of tuple(s)"
    assert isinstance(relative, bool), "relative must be a boolean"

    # Safety checks
    freqs = np.asarray(freqs)
    psd = np.asarray(psd)
    assert freqs.ndim == 1, "freqs must be a 1-D array of shape (n_freqs,)"
    assert psd.shape[-1] == freqs.shape[-1], "n_freqs must be last axis of psd"

    # Extract frequencies of interest
    all_freqs = np.hstack([[b[0], b[1]] for b in bands])
    fmin, fmax = min(all_freqs), max(all_freqs)
    idx_good_freq = np.logical_and(freqs >= fmin, freqs <= fmax)
    freqs = freqs[idx_good_freq]
    res = freqs[1] - freqs[0]

    # Trim PSD to frequencies of interest
    psd = psd[..., idx_good_freq]

    # Check if there are negative values in PSD
    if (psd < 0).any():
        msg = (
            "There are negative values in PSD. This will result in incorrect "
            "bandpower values. We highly recommend working with an "
            "all-positive PSD. For more details, please refer to: "
            "https://github.com/raphaelvallat/yasa/issues/29"
        )
        logger.warning(msg)

    # Calculate total power
    total_power = simps(psd, dx=res, axis=-1)
    total_power = total_power[np.newaxis, ...]

    # Initialize empty array
    bp = np.zeros((len(bands), *psd.shape[:-1]), dtype=np.float64)

    # Enumerate over the frequency bands
    labels = []
    for i, band in enumerate(bands):
        b0, b1, la = band
        labels.append(la)
        idx_band = np.logical_and(freqs >= b0, freqs <= b1)
        bp[i] = simps(psd[..., idx_band], dx=res, axis=-1)

    if relative:
        bp /= total_power
    return bp


# Compute bandpowers in N-dimensional PSD
bp = bandpower_from_psd_ndarray(psd, freqs)
bp = pd.DataFrame(bp.T, columns=["delta", "theta", "alpha", "sigma", "beta", "gamma"])
df_feat = pd.concat([df_feat, bp], axis=1)
df_feat.head()


In [ ]:
# Ratio of spectral power
df_feat.eval("dt = delta / theta", inplace=True)
df_feat.eval("da = delta / alpha", inplace=True)
df_feat.eval("ds = delta / sigma", inplace=True)
df_feat.eval("db = delta / beta", inplace=True)
df_feat.eval("db = delta / gamma", inplace=True)
df_feat.eval("ta = theta / alpha", inplace=True)
df_feat.eval("ts = theta / sigma", inplace=True)
df_feat.eval("tb = theta / beta", inplace=True)
df_feat.eval("tg = theta / gamma", inplace=True)
df_feat.eval("asi = alpha / sigma", inplace=True)
df_feat.eval("ab = alpha / beta", inplace=True)
df_feat.eval("ag = alpha / gamma", inplace=True)
df_feat.eval("at = alpha / theta", inplace=True)

df_feat.head()


# Compute the envelope derivative operator

In [ ]:
"""Compute the envelope derivative operator (EDO), as defined in [1].
[1] JM O' Toole, A Temko, NJ Stevenson, “Assessing instantaneous energy in the EEG: a non-negative, frequency-weighted energy operator”, IEEE Int. Conf.  on Eng. in Medicine and Biology, Chicago, August 2014
"""


def discrete_hilbert(x, DBplot=False):
    """Discrete Hilbert transform
    Parameters
    ----------
    x: ndarray
        input signal
    DBplot: bool, optional
        plot or not
    Returns
    -------
    x_hilb : ndarray
        Hilbert transform of x
    """
    N = len(x)
    Nh = np.ceil(N / 2)
    k = np.arange(N)

    # build the Hilbert transform in the frequency domain:
    H = -1j * np.sign(Nh - k) * np.sign(k)
    x_hilb = np.fft.ifft(np.fft.fft(x) * H)
    x_hilb = np.real(x_hilb)

    if DBplot:
        plt.figure(10, clear=True)
        plt.plot(np.imag(H))

    return x_hilb


def edo(x, DBplot=False):
    """Generate Envelope Derivative Operator (EDO) Γ[x(n)] from simple formula in the time domain:
    Γ[x(n)] = y(n)² + H[y(n)]²
    where y(n) is the derivative of x(n) using the central-finite method and H[.] is the
    Hilbert transform.
    Parameters
    ----------
    x: ndarray
        input signal
    DBplot: bool, optional
        plot or not
    Returns
    -------
    x_edo : ndarray
        EDO of x
    """
    # 1. check if odd length and if so make even:
    N_start = len(x)
    if (N_start % 2) != 0:
        x = np.hstack((x, 0))

    N = len(x)
    nl = np.arange(1, N - 1)
    xx = np.zeros(N)

    # 2. calculate the Hilbert transform
    h = discrete_hilbert(x)

    # 3. implement with the central finite difference equation
    xx[nl] = (
        (x[nl + 1] ** 2) + (x[nl - 1] ** 2) + (h[nl + 1] ** 2) + (h[nl - 1] ** 2)
    ) / 4 - ((x[nl + 1] * x[nl - 1] + h[nl + 1] * h[nl - 1]) / 2)

    # trim and zero-pad and the ends:
    x_edo = np.pad(xx[2 : (len(xx) - 2)], (2, 2), "constant", constant_values=(0, 0))

    if DBplot:
        plt.figure(2, clear=True)
        (hl1,) = plt.plot(x, label="test signal")
        (hl2,) = plt.plot(x_edo, label="EDO")
        plt.legend(handles=[hl1, hl2], loc="upper right")
        plt.pause(0.0001)

    return x_edo[0:N_start]


def test_edo_random():
    """Test EDO with a random signal"""

    DBplot = True
    x = np.random.randn(102)
    x_e = edo(x)

    # -------------------------------------------------------------------
    # plot
    # -------------------------------------------------------------------
    if DBplot:
        plt.figure(2, clear=True)
        (hl1,) = plt.plot(x, label="test signal")
        (hl2,) = plt.plot(x_e, label="EDO")
        plt.legend(handles=[hl1, hl2], loc="upper right")
        plt.pause(0.0001)


""" General_nleo: ''General'' Non-Linear Energy Operator (NLEO) expression: 
Ψ(n)=x(n-l)x(n-p)-x(n-q)x(n-s) for l+p=q+s  (and [l,p]≠[q,s], otherwise Ψ(n)=0)
"""


def gen_nleo(x, l=1, p=2, q=0, s=3):
    """general form of the nonlinear energy operator (NLEO)
    General NLEO expression: Ψ(n) = x(n-l)x(n-p) - x(n-q)x(n-s)
    for l+p=q+s  (and [l,p]≠[q,s], otherwise Ψ(n)=0)
    Parameters
    ----------
    x: ndarray
        input signal
    l: int, optional
        parameter of NLEO expression (see above)
    p: int, optional
        parameter of NLEO expression (see above)
    q: int, optional
        parameter of NLEO expression (see above)
    s: int, optional
        parameter of NLEO expression (see above)
    Returns
    -------
    x_nleo : ndarray
        NLEO array
    Example
    -------
    import numpy as np
    # generate test signal
    N = 256
    n = np.arange(N)
    w1 = np.pi / (N / 32)
    ph1 = -np.pi + 2 * np.pi * np.random.rand(1)
    a1 = 1.3
    x1 = a1 * np.cos(w1 * n + ph1)
    # compute instantaneous energy:
    x_nleo = gen_nleo(x1, 1, 2, 0, 3)
    # plot:
    plt.figure(1, clear=True)
    plt.plot(x1, '-o', label='test signal')
    plt.plot(x_nleo, '-o', label='Agarwal-Gotman')
    plt.legend(loc='upper left')
    """
    # check parameters:
    if (l + p) != (q + s) and any(np.sort((l, p)) != np.sort((q, s))):
        warning("Incorrect parameters for NLEO. May be zero!")

    N = len(x)
    x_nleo = np.zeros(N)

    iedges = abs(l) + abs(p) + abs(q) + abs(s)
    n = np.arange(iedges + 1, (N - iedges - 1))

    x_nleo[n] = x[n - l] * x[n - p] - x[n - q] * x[n - s]

    return x_nleo


def nleo(x, type="teager"):
    """generate different NLEOs based on the same operator
    Parameters
    ----------
    x: ndarray
        input signal
    type: {'teager', 'agarwal', 'palmu', 'abs_teager', 'env_only'}
        which type of NLEO?

    Returns
    -------
    x_nleo : ndarray
        NLEO array


    Additional Notes
    ----------------
    {'teager': 'Teager-Kaiser', 'agarwal': 'Agarwal-Gotman', 'palmu': 'Palmu et.al.'}

    """

    def teager():
        return gen_nleo(x, 0, 0, 1, -1)

    def agarwal():
        return gen_nleo(x, 1, 2, 0, 3)

    def palmu():
        return abs(gen_nleo(x, 1, 2, 0, 3))

    def abs_teager():
        return abs(gen_nleo(x, 0, 0, 1, -1))

    def env_only():
        return abs(x) ** 2

    def default_nleo():
        # -------------------------------------------------------------------
        # default option
        # -------------------------------------------------------------------
        print("Invalid NLEO name; defaulting to Teager")
        return teager()

    # pick which function to execute
    which_nleo = {
        "teager": teager,
        "agarwal": agarwal,
        "palmu": palmu,
        "abs_teager": abs_teager,
        "env_only": env_only,
    }

    def get_nleo(name):
        return which_nleo.get(name, default_nleo)()

    x_nleo = get_nleo(type)
    return x_nleo


def test_compare_nleos(x=None, DBplot=True):
    """test all NLEO variants with 1 signal
    Parameters
    ----------
    x: ndarray, optional
        input signal (defaults to coloured Gaussian noise)
    DBplot: bool
        plot or not
    """
    if x is None:
        N = 128
        x = np.cumsum(np.random.randn(N))

    all_methods = ["teager", "agarwal", "palmu"]
    all_methods_strs = {
        "teager": "Teager-Kaiser",
        "agarwal": "Agarwal-Gotman",
        "palmu": "Palmu et.al.",
    }
    x_nleo = dict.fromkeys(all_methods)

    for n in all_methods:
        x_nleo[n] = nleo(x, n)

    if DBplot:
        fig, ax = plt.subplots(nrows=2, ncols=1, num=4, clear=True)
        ax[0].plot(x, "-o", label="test signal")
        for n in all_methods:
            ax[1].plot(x_nleo[n], "-o", label=all_methods_strs[n])
        ax[0].legend(loc="upper right")
        ax[1].legend(loc="upper left")
        plt.pause(0.0001)


In [ ]:
def energy_operators_from_signal_ndarray(
    x, ops=["teager", "agarwal", "palmu", "abs_teager", "env_only", "edo"]
):
    def teager():
        return nleo(x, "teager")

    def agarwal():
        return nleo(x, "agarwal")

    def palmu():
        return nleo(x, "palmu")

    def abs_teager():
        return nleo(x, "abs_teager")

    def env_only():
        return nleo(x, "env_only")

    def edo_f():
        return edo(x)

    def default_eo():
        # -------------------------------------------------------------------
        # default option
        # -------------------------------------------------------------------
        print("Invalid EO name; defaulting to Teager")
        return teager()

    which_energy_operator = {
        "teager": teager,
        "agarwal": agarwal,
        "palmu": palmu,
        "abs_teager": abs_teager,
        "env_only": env_only,
        "edo": edo_f,
    }

    def get_energy_operator(name):
        return which_energy_operator.get(name, default_eo)()

    feat = np.zeros((x.shape[0], len(ops)))

    for i, op in enumerate(ops):
        x_nleo = get_energy_operator(op)  # Function
        feat[:, i] = x_nleo

    return feat


data_win_mean = apply(np.mean, axis=1, arr=data_win)
featEnergy = energy_operators_from_signal_ndarray(
    data_win_mean, ops=["teager", "agarwal", "palmu", "abs_teager", "env_only", "edo"]
)
featEnergy = pd.DataFrame(
    featEnergy, columns=["teager", "agarwal", "palmu", "abs_teager", "env_only", "edo"]
)
df_feat = pd.concat([df_feat, featEnergy], axis=1)
df_feat.head()


In [ ]:
# Write feature object to a comma-separated values (csv) file
df_feat.to_csv(f"feature/{fname} {lr}.csv", index=False)


In [4]:
# Load feature object as a dataframe
df_feat = pd.read_csv(f"feature/{fname} {lr}.csv", index_col=False)


In [5]:
# Describe
df_feat.agg(["min", "median", "max", "std"]).round(2).T


,min,median,max,std
std,3.26,15.71,230.73,18.54
mean,-8.49,0.00,10.43,0.64
median,-5.64,0.26,7.25,1.40
iqr,4.28,15.11,82.69,11.92
skew,-5.26,-0.20,6.16,0.92
kurt,-0.81,1.15,55.75,6.90
nzc,43.00,209.50,578.00,128.26
hmob,0.02,0.07,0.50,0.06
hcomp,1.69,7.88,22.33,3.60
perm_entropy,0.62,0.70,0.80,0.04


In [ ]:
import seaborn as sns

feat_set1 = ["std", "mean", "median", "iqr", "skew", "kurt", "nzc", "hmob", "hcomp"]

feat_set2 = [
    "perm_entropy",
    "svd_entropy",
    "sample_entropy",
    "app_entropy",
    "spec_entropy",
    "dfa",
    "petrosian",
    "katz",
    "higuchi",
]

# Distribution
plt.figure()
df_feat[feat_set1].hist(figsize=(6, 7), layout=(3, 3), sharey=True)
plt.tight_layout()
plt.show()

df_feat[feat_set2].hist(figsize=(6, 7), layout=(3, 3), sharey=True)
plt.tight_layout()
plt.show()

# Heatmap
plt.figure(figsize=(8, 8))
sns.heatmap(
    df_feat[feat_set1].corr().round(2),
    annot=True,
    vmin=-1,
    vmax=1,
    cbar=False,
    cmap="Spectral_r",
    square=True,
)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 8))
sns.heatmap(
    df_feat[feat_set2].corr().round(2),
    annot=True,
    vmin=-1,
    vmax=1,
    cbar=False,
    cmap="Spectral_r",
    square=True,
)
plt.tight_layout()
plt.show()

# Dengrogram using Agglomerative(bottom-up) hierarchical clustering
# In Agglomerative clustering, we start with considering each data point as a cluster and then repeatedly combine two nearest clusters into larger clusters until we are left with a single cluster
# Coming to the heat map, it is a graphical representation of data where values are represented using colors. Variation in the intensity of color depicts how data is clustered or varies over space.
plt.figure()
sns.clustermap(df_feat[feat_set1].interpolate(), row_cluster=False, standard_scale=1)
plt.tight_layout()
plt.show()

plt.figure()
sns.clustermap(df_feat[feat_set2].interpolate(), row_cluster=False, standard_scale=1)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.feature_selection import f_classif

# Load hypnogram
location_hypno = "/Users/amirhosseindaraie/Desktop/data/synced-hypnos"
hypno_30s = np.loadtxt(f"{location_hypno}/p8n3_synced.txt")[:, 0]

# Extract sorted F-values
# Compute the ANOVA F-value for the provided sample.
fvals = pd.Series(
    f_classif(X=df_feat, y=hypno_30s)[0], index=df_feat.columns
).sort_values()

# Plot features ranking
plt.figure(figsize=(6, 6))
sns.barplot(y=fvals.index, x=fvals, palette="RdYlGn")
plt.xlabel("F-values")
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()


In [ ]:
# Plot hypnogram and a feature
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

hypno = pd.Series(hypno_30s).map({-1: -1, 0: 0, 1: 2, 2: 3, 3: 4, 4: 1}).values
hypno_rem = np.ma.masked_not_equal(hypno, 1)

# Plot the hypnogram
ax1.step(times, -1 * hypno, color="k", lw=1.5)
ax1.step(times, -1 * hypno_rem, color="r", lw=2.5)
ax1.set_yticks([0, -1, -2, -3, -4])
ax1.set_yticklabels(["W", "R", "N1", "N2", "N3"])
ax1.set_ylim(-4.5, 0.5)
ax1.set_ylabel("Sleep stage")

# Plot the non-linear feature
ax2.plot(times, df_feat["perm_entropy"])
ax2.set_ylabel("Permutation Entropy")
# ax2.set_ylabel('Higuchi Fractal Dimension')
ax2.set_xlabel("Time [minutes]")

ax2.set_xlim(0, times[-1])

plt.tight_layout()
plt.show()


In [ ]:
# Plot hypnogram and 4 top features
fig, axs = plt.subplots(5, 1, figsize=(12, 6), sharex=True)

hypno = pd.Series(hypno_30s).map({-1: -1, 0: 0, 1: 2, 2: 3, 3: 4, 4: 1}).values
hypno_rem = np.ma.masked_not_equal(hypno, 1)

# Plot the hypnogram
axs[0].step(times, -1 * hypno, color="k", lw=1.5)
axs[0].step(times, -1 * hypno_rem, color="r", lw=2.5)
axs[0].set_yticks([0, -1, -2, -3, -4])
axs[0].set_yticklabels(["W", "R", "N1", "N2", "N3"])
axs[0].set_ylim(-4.5, 0.5)
axs[0].set_ylabel("Sleep stage")

# Select top features and normalize to 1
topFeats = fvals.iloc[::-1][0:4].keys().tolist()
df_topFeats = df_feat[topFeats]
df_topFeats = df_topFeats.apply(lambda x: x / x.max(), axis=0)

# Plot the non-linear feature
cmap = ["blue", "orange", "green", "red"]
for i, (featName, featData) in enumerate(df_topFeats.iteritems()):
    axs[i + 1].plot(times, featData, c=cmap[i])
    axs[i + 1].set_ylabel(featName)

axs[-1].set_xlabel("Time [minutes]")
axs[-1].set_xlim(0, times[-1])

plt.tight_layout()
plt.show()


In [ ]:
plt.plot(df_feat["mean"])
# plt.plot(df_feat["teager"])
plt.plot(df_feat["edo"])